# Professor Adams - Financial Risk Management (FRE 6123) 
# Homework #1

### README
- Part 1: Setup, libraries, helper functions.
- Part 2:
- Q1: Annualized returns; arithmetic vs geometric means; std; R/σ; leverage comparison.
- Q2: Archegos case mapped to Stulz’s risk-management failures.
- Q3: Continuously compounded monthly returns; population stats; covariance/correlation.
- Q4: SPY 2019–2020 daily log-returns; annualized Sharpe/Sortino; 1-day HS VaR.
- Q5: Bond portfolio duration/convexity; impacts under steepening; curvature-tilted portfolio.

---
---

### Part 1 Set up Environment

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import yfinance as yf
from typing import Any

In [ ]:
# Define helper functions for reusable purposes 
# Spreadsheet functions will be displayed below

# Varying period length (c periods per year):
def var_annual(rates, periods_per_year):
    # Unwrap one level if a single nested list is passed, e.g., [Madsden]
    if isinstance(rates, (list, tuple, np.ndarray)) and len(rates) == 1 and isinstance(rates[0], (list, tuple, np.ndarray)):
        rates = rates[0]

    arr = np.asarray(rates, dtype=float)
    if arr.ndim == 0:
        arr = np.asarray([arr], dtype=float)

    # Auto-detect percent inputs (>1 or <-1): treat as percent, convert to decimal
    arr = np.where(np.abs(arr) > 1.0, arr / 100.0, arr)

    annualized = (1.0 + arr) ** periods_per_year - 1.0
    return [f"{float(x) * 100.0:.2f}%" for x in np.ravel(annualized)]
    
# Geometric Mean
def geo_mean(values, as_percent=True, decimals=2):
    nums = []
    for v in values:
        if isinstance(v, str):
            v = v.strip()
            if v.endswith('%'):
                v = float(v[:-1]) / 100.0
            else:
                v = float(v)
                if abs(v) > 1: v = v / 100.0
        else:
            v = float(v)
            if abs(v) > 1: v = v / 100.0
        nums.append(v)
    a = np.asarray(nums, dtype=float)
    if a.size == 0: return 'nan%' if as_percent else float('nan')
    if (a <= -1.0).any(): raise ValueError("Exists <= -100%, geometric mean is undefined")
    g = float(np.prod(1.0 + a)**(1.0/a.size) - 1.0)
    return f"{g*100:.{decimals}f}%" if as_percent else g


# Arithmetic Mean
def arith_mean(values, as_percent=True, decimals=2):
    nums = []
    for v in values:
        if isinstance(v, str):
            s = v.strip()
            if s.endswith('%'):
                val = float(s[:-1]) / 100.0
            else:
                val = float(s)
                if abs(val) > 1.0:  # interpret as percent if magnitude > 1
                    val /= 100.0
        else:
            val = float(v)
            if abs(val) > 1.0:      # interpret as percent if magnitude > 1
                val /= 100.0
        nums.append(val)

    a = np.asarray(nums, dtype=float)
    if a.size == 0:
        return 'nan%' if as_percent else float('nan')

    m = float(np.mean(a))
    return f"{m * 100:.{decimals}f}%" if as_percent else round(m, decimals)

# Leverage
def lever(r, rB=0.05):
    def to_decimal(x):
        if isinstance(x, str):
            s = x.strip()
            if s.endswith('%'): return float(s[:-1]) / 100.0
            v = float(s); return v/100.0 if abs(v) > 1.0 else v
        v = float(x); return v/100.0 if abs(v) > 1.0 else v
    return [f"{(2*to_decimal(x) - rB)*100:.2f}%" for x in r]

# Automatic Calculation of sample means & Standard Deviation & R/σ ratio for Question 1
def q1_metrics(df):
    df_num = df.replace('%','',regex=True).apply(pd.to_numeric, errors='coerce')/100
    R = df_num.mean()
    G = (1+df_num).prod()**(1/df_num.shape[0]) - 1
    S = df_num.std(ddof=1)
    return pd.DataFrame({'Arithmetic_Mean':R,'Geometric_Mean':G,'Sigma':S,'R/σ ratio':R/S})

# Sortino function
def sortino(series, mar=0.0):
    import numpy as np, pandas as pd
    s = pd.Series(series).astype(str)
    s = pd.to_numeric(s.str.rstrip('%'), errors='coerce')
    if s.abs().max() > 1: s = s/100.0  
    if isinstance(mar, str) and mar.strip().endswith('%'):
        mar = float(mar.strip()[:-1])/100.0
    elif abs(float(mar)) > 1: mar = float(mar)/100.0
    r = s.to_numpy(dtype=float)
    mean_r = r.mean()
    downside = np.minimum(0.0, r - mar)
    dd = float(np.sqrt(np.mean(downside**2)))
    srt = (mean_r - mar)/dd if dd > 0 else np.nan
    return mean_r, dd, srt

# Question 4 helper functions
def fetch_adjclose(ticker, start, end):
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    return df.rename(columns={'Adj Close':'AdjClose'})['AdjClose'].dropna()

def log_returns(prices: pd.Series) -> pd.Series:
    # r_t = ln(P_t / P_{t-1})
    return np.log(prices / prices.shift(1)).dropna()

def sharpe_ann(r_log: pd.Series, rf_daily=RF_DAILY, td=T_DAYS) -> float:
    mu = r_log.mean()
    sd = r_log.std(ddof=1)
    return ((mu - rf_daily) / sd) * np.sqrt(td)

def sortino_ann(r_log: pd.Series, mar=RF_DAILY, td=T_DAYS) -> float:
    mu = r_log.mean()
    downside = np.minimum(0.0, r_log - mar)
    dd = np.sqrt(np.mean(downside**2))        # population downside deviation
    return ((mu - mar) / dd) * np.sqrt(td)

def var_hs_1d(r_simple: pd.Series, alpha=0.95) -> float:
    # HS VaR at 95%: positive loss threshold per $1
    return float(-np.quantile(r_simple.dropna(), 1 - alpha)) 

---
---

### Part 2 Answers

#### Question 1 
#### a.  

In [134]:
Madsden = [ 1.2, 2.5, -1.50]
Quaker = [ 2.75, 3.2, -1.10]
Aldrige = [ -15, 40, 2.5] 

# varying period length (c periods per year):
Madsden_annual = var_annual([Madsden],12)
Quaker_annual = var_annual([Quaker],4)
Aldrige_annual = var_annual([Aldrige],1)

# quick check
df_q1a = pd.DataFrame({ 'Madsden': Madsden_annual, 
                        'Quaker': Quaker_annual,
                        'Aldrige': Aldrige_annual
                    })
df_q1a.columns.name = 'Inverstment Annual Returns'
df_q1a.index = ['Year 1', 'Year 2', 'Year 3']
df_q1a

Inverstment Annual Returns,Madsden,Quaker,Aldrige
Year 1,15.39%,11.46%,-15.00%
Year 2,34.49%,13.43%,40.00%
Year 3,-16.59%,-4.33%,2.50%


In [135]:
# Arithmetic versus Geometric Mean
Madsden_geo = geo_mean(Madsden_annual)
Quaker_geo = geo_mean(Quaker_annual)
Aldrige_geo = geo_mean(Aldrige_annual)
Madsden_arith = arith_mean(Madsden_annual)
Quaker_arith = arith_mean(Quaker_annual)
Aldrige_arith = arith_mean(Aldrige_annual)

# quick check
df_q1a_mean = pd.DataFrame([{
    'Madsden_geo': Madsden_geo,
    'Quaker_geo': Quaker_geo,
    'Aldrige_geo': Aldrige_geo,
    'Madsden_arith': Madsden_arith,
    'Quaker_arith': Quaker_arith,
    'Aldrige_arith': Aldrige_arith
}])
df_q1a_mean.index = ['Mean']
df_q1a_mean


,Madsden_geo,Quaker_geo,Aldrige_geo,Madsden_arith,Quaker_arith,Aldrige_arith
Mean,8.98%,6.55%,6.85%,11.10%,6.85%,9.17%


In [136]:
# print the highest Arithmetic & Geometric Mean
print("The highest Arithmetic Mean: " + "Madsden_arith" +" " + str(Madsden_arith))
print("The highest Geometric Mean: " + "Madsden_geo" +" " + str(Madsden_geo))

The highest Arithmetic Mean: Madsden_arith 11.10%
The highest Geometric Mean: Madsden_geo 8.98%


#### Arithmetic vs Geometric Mean (Why They Differ)
- **Definition**: Arithmetic \(a=\frac{1}{n}\sum r_t\); Geometric \(g=\left(\prod (1+r_t)\right)^{1/n}-1\).
- **Compounding vs. averaging**: Geometric mean captures multi-period compounding; arithmetic is a simple per-period average.
- **Volatility drag**: More volatility lowers \(g\). Approximation (small returns): \(g \approx a - \frac{1}{2}\sigma^2\).
- **Negative-return asymmetry**: A −50% drop needs +100% to break even—reflected in \(g\), not in \(a\).
- **Use cases**: Use arithmetic for single-period expectations; geometric for long-run growth. Align frequencies (e.g., annualize) before comparing.

---

#### Question 1 
#### b.

In [137]:
# Standard deviation 
Madsden_annual_std = np.std(np.array([float(x.strip('%')) for x in Madsden_annual])/100.0, ddof=1)
Quaker_annual_std = np.std(np.array([float(x.strip('%')) for x in Quaker_annual])/100.0, ddof=1)
Aldrige_annual_std = np.std(np.array([float(x.strip('%')) for x in Aldrige_annual])/100.0, ddof=1)

# Quick check
df_q1b_std = pd.DataFrame ([{
    'Madsden_std': Madsden_annual_std,
    'Quaker_std': Quaker_annual_std,
    'Aldrige_std': Aldrige_annual_std
}])
df_q1b_std.index = ['Standard Deviation']
df_q1b_std

,Madsden_std,Quaker_std,Aldrige_std
Standard Deviation,0.258092,0.097350,0.280995


+ Note：Higher std ⇒ higher risk: Typically indicates more uncertainty and larger swings in performance.

In [138]:
# ranking
df_q1b_ranks = pd.DataFrame(
    {'Investment': ['Aldrige','Madsden','Quaker'],
     'Ranking': [Aldrige_annual_std, Madsden_annual_std, Quaker_annual_std]},
    index=['highest risk','medium risk','lowest risk']
)
df_q1b_ranks

,Investment,Ranking
highest risk,Aldrige,0.280995
medium risk,Madsden,0.258092
lowest risk,Quaker,0.097350


---

#### Question 1 
#### c.

In [139]:
# Calculates R/sigma ratio

df_q1c = df_q1a.replace('%', '', regex=True).apply(pd.to_numeric, errors='coerce') / 100
R = df_q1c.mean() # arithmetic average annual return
sigma_σ = df_q1c.std(ddof=1) # sample std dev = sigma
ratio = R / sigma_σ

q1c_summary = pd.DataFrame({
    "arithmetic_mean": R,
    "sample_std_dev":sigma_σ,
    "R/σ_Ratio": ratio
}).round(6)

ranking = ratio.sort_values(ascending=False).index.tolist()

print(q1c_summary.map('{:.2%}'.format).to_string())
print()
print("Ranking (best to worst) by R/σ_Ratio:", " > ".join(ranking))

                           arithmetic_mean sample_std_dev R/σ_Ratio
Inverstment Annual Returns                                         
Madsden                             11.10%         25.81%    42.99%
Quaker                               6.85%          9.74%    70.40%
Aldrige                              9.17%         28.10%    32.62%

Ranking (best to worst) by R/σ_Ratio: Quaker > Madsden > Aldrige


---

#### Question 1 
#### d.

In [140]:
# caculate the levered versus unlevered R/σ ratio
rB = 0.05 # borrowed interest rate
L  = 1.0 # Vb/Ve = 1

lev_df = pd.DataFrame({
    "Madsden_L":  lever(Madsden_annual),
    "Quaker_L":   lever(Quaker_annual),
    "Aldridge_L": lever(Aldrige_annual),
})

lev_metrics = q1_metrics(lev_df).round(6)
ranking_unlev = q1_metrics(lev_df).round(6).sort_values(by='R/σ ratio', ascending=False).index.tolist()

print(q1c_summary.map('{:.2%}'.format).to_string())
print()
print("\nLevered metrics (50% borrowed @5%):")
print(lev_metrics.map('{:.2%}'.format).to_string())
print("\nRanking by R/σ (unlevered):", " > ".join(ranking))
print("Ranking by R/σ (levered):", " > ".join(ranking_unlev))

                           arithmetic_mean sample_std_dev R/σ_Ratio
Inverstment Annual Returns                                         
Madsden                             11.10%         25.81%    42.99%
Quaker                               6.85%          9.74%    70.40%
Aldrige                              9.17%         28.10%    32.62%


Levered metrics (50% borrowed @5%):
           Arithmetic_Mean Geometric_Mean   Sigma R/σ ratio
Madsden_L           17.19%          8.44%  51.62%    33.31%
Quaker_L             8.71%          7.45%  19.47%    44.72%
Aldridge_L          13.33%          4.39%  56.20%    23.73%

Ranking by R/σ (unlevered): Quaker > Madsden > Aldrige
Ranking by R/σ (levered): Quaker_L > Madsden_L > Aldridge_L


#### Answer: Why (a)–(c) change with 50% leverage at 5%

**Setup.** With 50% debt ($D/E=1$) and borrowing rate $r_B=5\%$, each year’s **levered equity return** is

$$
\boxed{r_L=(1+\tfrac{D}{E})\,r-\tfrac{D}{E}\,r_B=2r-0.05 }.
$$

#### (a) Averages

* **Arithmetic mean:** $\displaystyle R_L=2R-0.05$. If the unlevered $R>2.5\%$, arithmetic mean rises; otherwise it falls.
* **Geometric mean:** leverage **raises volatility**, increasing variance drag (rule-of-thumb $G\approx R-\tfrac12\sigma^2$). Since $\sigma$ doubles (see below), the drag scales to $0.5(2\sigma)^2=2\sigma^2$, so $G$ typically **declines** despite higher arithmetic mean.

#### (b) Risk

Because $r_L=2r-0.05$ is a linear transformation, the constant $-0.05$ doesn’t affect dispersion and

$$
\boxed{\sigma_L=2\sigma}.
$$

#### (c) Simple return-vs-risk ratio $R/\sigma$

$$
\frac{R_L}{\sigma_L}=\frac{2R-0.05}{2\sigma}
=\underbrace{\frac{R}{\sigma}}_{\text{original}}-\underbrace{\frac{0.05}{2\sigma}}_{\text{penalty from financing cost}}.
$$

Thus **$R/\sigma$ falls for all three**, and assets with **smaller $\sigma$** take a **larger penalty** (the $1/\sigma$ term). Rankings **may** change; in this dataset they remain **Quaker (L) > Madsden (L) > Aldridge (L)**, but with lower $R/\sigma$ across the board.

**Intuition:** Leverage magnifies gains and losses; the fixed borrowing cost plus higher volatility reduce risk-adjusted performance and long-run compounding even when arithmetic returns rise.


---

#### Question 1 
#### e.

**Simple Sharpe (This Question):**

$$
\boxed{\text{Sharpe}_{\text{simple}}=\frac{R}{\sigma}}
$$

**Sortino:**

$$
\boxed{\text{Sortino}=\frac{\overline{R}-\mathrm{MAR}}{\sigma_D}},\qquad
\boxed{\sigma_D=\sqrt{\frac{1}{n}\sum_{t=1}^{n}\min\!\bigl(0,\; r_t-\mathrm{MAR}\bigr)^2}}
$$

**Where**

* $R$ (or $\overline{R}$): mean return (arithmetic),
* $\sigma$: standard deviation of returns (total volatility),
* $\mathrm{MAR}$: minimum acceptable return,
* $\sigma_D$: downside deviation,
* $r_t$: period-$t$ return, $n$: number of periods.


In [141]:
# Calculate the Sortino ratio
MAR = 0.0  # minimum acceptable return
rows = {}
for col in df_q1a.columns:
    R, DD, S = sortino(df_q1a[col], MAR)
    rows[col] = {"Mean_Return_R": R, "Downside_Deviation": DD, "Sortino": S}

res = pd.DataFrame(rows).T.round(6)
res["Rank_Sortino"] = res["Sortino"].rank(ascending=False, method="min").astype(int)
print(res.to_string())

         Mean_Return_R  Downside_Deviation  Sortino  Rank_Sortino
Madsden       0.110967            0.095782 1.158529             2
Quaker        0.068533            0.024999 2.741414             1
Aldrige       0.091667            0.086603 1.058475             3


**Interpretion**
: 
- Quaker has the highest Sortino because its downside deviation is very small (only a mild negative year), so it delivers the best return per unit of downside risk. Madsden and Aldridge each have a deeper negative year, increasing downside deviation and lowering their Sortino ratios.

---
---

#### Question 2

#### Archegos Case and Stulz’s Risk Management Failures

René Stulz (2008) outlines six common failures of risk management:

1. Accurately measure known risks
2. Adequately take risks into account
3. Communicate risks to top management
4. Adequately and consistently monitor risks
5. Manage risks properly
6. Use appropriate risk metrics

In the Archegos case, at least three failures were evident:

##### 1. Failure to Accurately Measure Known Risks

The Credit Suisse report noted that “Archegos began regularly breaching its potential exposure limit in the spring of 2020,” with exposures many times above the approved threshold. Yet, risk teams discounted these breaches after changes in methodology and allowed them to persist. This shows that known risks were mismeasured or disregarded rather than properly incorporated.



##### 2. Failure to Adequately Take Risks into Account

According to the investigation, Archegos was “monitored under a more forgiving ‘Bad Week’ scenario” instead of the more severe stress tests normally applied. By choosing lenient assumptions, Credit Suisse underestimated the true risk of Archegos’s concentrated positions. This demonstrates that even when risks were measured, they were not adequately taken into account in decision-making.



##### 3. Failure to Communicate Risks to Top Management

At the September 2020 Counterparty Oversight Committee meeting, materials observed that Archegos “makes substantial use of leverage” and had “chunky single-name stock exposures.” However, the committee took no urgent action, deciding only to “revisit the counterparty at a future meeting.” Risks were therefore acknowledged but not escalated effectively to senior management.



#### Conclusion

The Archegos collapse illustrates several failures in Stulz’s typology. Credit Suisse failed to **measure risks accurately**, did not **take known risks seriously**, and failed to **communicate and escalate** exposures. These weaknesses directly contributed to losses of over \$5.5 billion when Archegos defaulted.

---
---

#### Question 3
#### a.

#### [Explanation]: Continuously Compounded Monthly Returns

We are given month-end prices (no cash flows) for Asset A and Asset B.
The **continuously compounded monthly return** is defined as

$$
r_t \;=\; \ln\!\left(\frac{P_t}{P_{t-1}}\right),
$$

where $P_t$ is the month-end price at month $t$ and $\ln(\cdot)$ is the natural logarithm.

**Procedure (what the next code cell will do):**

1. Build the price table for months $0 \ldots 12$.
2. Compute monthly log returns using $r_t=\ln(P_t/P_{t-1})$ for $t=1\ldots12$.
3. Report the **mean** of those monthly log returns for each asset.


In [146]:
# calculate the continuously compounded monthly price return

q3 = pd.DataFrame({
    "Month": list(range(0, 13)),
    "Asset_A": [24.9, 25.1, 23.37, 24.75, 26.62, 26.5, 28.0, 28.88, 30.0, 31.38, 36.25, 37.13, 36.88],
    "Asset_B": [43.9, 44.85, 46.88, 45.25, 47.0, 58.5, 57.25, 62.75, 65.5, 74.38, 78.5, 78.0, 78.12],
}).set_index("Month")

# --- Compute continuously compounded monthly returns ---
# r_t = ln(P_t / P_{t-1})
log_rets = np.log(q3 / q3.shift(1)).iloc[1:]   # drop month 0 (nan)

# --- Summary: mean continuously compounded monthly returns for each asset ---
mean_log_rets = log_rets.mean()

# --- Quick Check ---
pd.options.display.float_format = "{:.6f}".format
print("Continuously compounded monthly returns (r_t = ln(P_t / P_{t-1})):\n")
print(log_rets.map('{:.2%}'.format))
print("\nContinuously compounded monthly returns:")
print(mean_log_rets.map('{:.2%}'.format))

Continuously compounded monthly returns (r_t = ln(P_t / P_{t-1})):

      Asset_A Asset_B
Month                
1       0.80%   2.14%
2      -7.14%   4.43%
3       5.74%  -3.54%
4       7.28%   3.79%
5      -0.45%  21.89%
6       5.51%  -2.16%
7       3.09%   9.17%
8       3.80%   4.29%
9       4.50%  12.71%
10     14.43%   5.39%
11      2.40%  -0.64%
12     -0.68%   0.15%

Continuously compounded monthly returns:
Asset_A    3.27%
Asset_B    4.80%
dtype: object


---

#### Question 3
#### b.

In [143]:
# Calculate the monthly variance and standard deviation for each asset
# Assuming population mean and variance in your calculations.

# --- Use (a)'s mean as E[r_i] (population mean) based on question assumption ---
E_ri = log_rets.mean()

# --- Population variance and standard deviation ---
# Var_pop = mean( (r_t - E[r_i])^2 ), Std_pop = sqrt(Var_pop)
var_pop = ((log_rets - E_ri)**2).mean()
std_pop = np.sqrt(var_pop)

# --- Qucik Check ---
q3_b = pd.DataFrame({
    "E_ri_from_a": E_ri,
    "Var_population": var_pop,
    "Std_population": std_pop
})
q3_b

,E_ri_from_a,Var_population,Std_population
Asset_A,0.032733,0.002485,0.049848
Asset_B,0.048028,0.004598,0.067806


---
#### Question 3
#### b.

#### Covariance & Correlation interpretation

Let monthly **log returns** be $r_{At}, r_{Bt}$ for $t=1,\dots,M$.
Using **population** statistics (as required):

$$
\boxed{\;\operatorname{Cov}(r_A,r_B)=\frac{1}{M}\sum_{t=1}^{M}
\big[r_{At}-\mathbb{E}(r_A)\big]\big[r_{Bt}-\mathbb{E}(r_B)\big]\;}
$$

$$
\boxed{\;\rho_{AB}=\dfrac{\operatorname{Cov}(r_A,r_B)}{\sigma_A\,\sigma_B}\;},\qquad
\sigma_A=\sqrt{\tfrac{1}{M}\sum_{t=1}^{M}(r_{At}-\mathbb{E}(r_A))^2},\;
\sigma_B=\sqrt{\tfrac{1}{M}\sum_{t=1}^{M}(r_{Bt}-\mathbb{E}(r_B))^2}.
$$

In [154]:
# Population means E[r_i] (from part a)
Ea = log_rets['Asset_A'].mean()
Eb = log_rets['Asset_B'].mean()

# Population standard deviations
sigmaA = np.sqrt(np.mean((log_rets["Asset_A"] - Ea)**2))
sigmaB = np.sqrt(np.mean((log_rets["Asset_B"] - Eb)**2))

# Population covariance and correlation
cov_pop = np.mean((log_rets["Asset_A"] - Ea) * (log_rets["Asset_B"] - Eb))
rho_ab  = cov_pop / (sigmaA * sigmaB)

# Qucik Check
q3_b = pd.DataFrame(
    {'Value': [Ea, sigmaA, Eb, sigmaB, cov_pop, rho_ab]},
    index=['E[r_A]', 'sigma_A (population)', 'E[r_B]', 'sigma_B (population)', 'Cov_population(A,B)', 'Corr(A,B)']
)

q3_b

,Value
E[r_A],0.032733
sigma_A (population),0.049848
E[r_B],0.048028
sigma_B (population),0.067806
"Cov_population(A,B)",-0.000458
"Corr(A,B)",-0.135545


---
#### Question 3
#### d. Interpretation 

* The covariance is slightly **negative** and the correlation is about **−0.136**, indicating a **weak inverse relationship** between monthly price changes in A and B.
* Practically: they tend to move in opposite directions a little more often than not, offering **modest diversification benefits** when combined in a portfolio.

---
---

#### Question 4
#### For this question, I will put the answers all in one dataframe


#### SPY Performance (2019 & 2020)
 - Fetch daily **Adjusted Close** with `yfinance`
 - (a) Compute **daily log-returns**
 - (b) Compute **annualized Sharpe** (rf = 0.5% p.a., 252 days)
 - (c) Compute **annualized Sortino** (MAR = rf_daily)
 - (d) Estimate **1-day Historical Simulation VaR (95%)** for each year

#### Notes:
 - Sharpe/Sortino are annualized from daily stats using sqrt(252).
 - VaR is *1-day* HS VaR computed from **simple** returns. If you need a
 - multi-day horizon, aggregate or apply a scaling assumption explicitly.

In [192]:
T_DAYS = 252
RF_ANNUAL = 0.005
RF_DAILY  = RF_ANNUAL / T_DAYS

periods = {
    "2019": ("2019-01-01", "2019-12-31"),
    "2020": ("2020-01-01", "2020-12-31"),
}

summary_rows = []
logret_store = {}

for yr, (start, end) in periods.items():
    adj = fetch_adjclose("SPY", start, end)
    r_log = log_returns(adj)
    r_simple = adj.pct_change().dropna()

    logret_store[yr] = r_log  # (a) store/log-returns per year

    sharpe = sharpe_ann(r_log)
    sortino = sortino_ann(r_log)
    var95 = var_hs_1d(r_simple, 0.95)

    summary_rows.append({
        "Year": yr,
        "Sample Total": len(r_log),
        "Mean_Daily_LogRet": r_log.mean(),
        "Std_Daily_LogRet": r_log.std(ddof=1),
        "Sharpe_ratio": sharpe,
        "Sortino_ratio": sortino,
        "VaR_95%_1d": var95
    })

In [193]:
# (a) Daily log-returns for each year
log_returns_df = pd.DataFrame({"2019": [logret_store["2019"]], "2020": [logret_store["2020"]]})
print("Daily log-returns (NaNs omitted):")
print(log_returns_df.dropna(how="all").round(6).to_string())

Daily log-returns (NaNs omitted):
                                                                                                                                                                                                                                                                                                       2019                                                                                                                                                                                                                                                                                                      2020
0  Ticker           SPY
Date                
2019-01-03 -0.024152
2019-01-04  0.032947
2019-01-07  0.007854
2019-01-08  0.009351
2019-01-09  0.004663
...              ...
2019-12-23  0.001526
2019-12-24  0.000031
2019-12-26  0.005309
2019-12-27 -0.000248
2019-12-30 -0.005529

[250 rows x 1 columns]  Ticker           SPY
Date                
2020-01-03 -0.007601
2020

In [194]:
# (b)(c)(d) Summary table
summary = pd.DataFrame(summary_rows).set_index("Year").round(6)
print("\nSummary metrics:")
summary


Summary metrics:


,Sample Total,Mean_Daily_LogRet,Std_Daily_LogRet,Sharpe_ratio,Sortino_ratio,VaR_95%_1d
Year,,,,,,
2019,250,Ticker SPY 0.001073 dtype: float64,Ticker SPY 0.007924 dtype: float64,Ticker SPY 2.110086 dtype: float64,Ticker SPY 3.033820 dtype: float64,0.012048
2020,251,Ticker SPY 0.000613 dtype: float64,Ticker SPY 0.021272 dtype: float64,Ticker SPY 0.442898 dtype: float64,Ticker SPY 0.589789 dtype: float64,0.031764


### Summary of Results
- **Sample Total**: Number of trading days in the year.
- **Mean_Daily_LogRet**: Average daily log return.
- **Std_Daily_LogRet**: Volatility (standard deviation) of daily log returns.
- **Sharpe_ratio**: Risk-adjusted return (using daily mean/vol) annualized; higher is better.
- **Sortino_ratio**: Like Sharpe but penalizes only downside volatility; higher is better.
- **VaR_95%_1d**: 1-day 95% historical Value at Risk (expected loss per $1 on a bad day).

### Interpretation
- **2019**: Lower volatility (~0.79% daily) with much higher Sharpe (~2.11) and Sortino (~3.03) → stronger risk-adjusted performance and smaller 1-day VaR (~1.20%).
- **2020**: Higher volatility (~2.13% daily) with lower Sharpe (~0.44) and Sortino (~0.59); larger 1-day VaR (~3.18%) → substantially higher downside risk.

---
---
#### Question 5

| Tenor | Coupon | Price | ModDur | Convexity |
| ----- | ------ | ----- | ------ | --------- |
| 2y    | 4.5    | 100   | 1.873  | 3.6       |
| 5y    | 5.0    | 100   | 4.329  | 19.5      |
| 10y   | 5.5    | 100   | 7.538  | 58.5      |


In [195]:
# Bond data
bonds = pd.DataFrame({
    "Tenor": ["2y", "5y", "10y"],
    "Price": [100, 100, 100],
    "ModDur": [1.873, 4.329, 7.538],
    "Convexity": [3.6, 19.5, 58.5]
}).set_index("Tenor")

# ---- a) Portfolio duration & convexity ----
# 1) Equally price-weighted portfolio of all three
weights_eq = np.array([1/3, 1/3, 1/3])

# 2) Bullet: fully in 5-year
weights_bullet = np.array([0, 1, 0])

# 3) Barbell: 50% in 2y and 50% in 10y
weights_barbell = np.array([0.5, 0, 0.5])

def port_stats(weights, bonds):
    dur = np.dot(weights, bonds["ModDur"])
    conv = np.dot(weights, bonds["Convexity"])
    return dur, conv

dur_eq, conv_eq = port_stats(weights_eq, bonds)
dur_bullet, conv_bullet = port_stats(weights_bullet, bonds)
dur_barbell, conv_barbell = port_stats(weights_barbell, bonds)

# ---- b) Impact of yield curve steepening ----
# Price change approximation: ΔP/P ≈ -Dur*Δy + 0.5*Conv*(Δy)^2
# (Δy expressed in decimals)

dY = {"2y": -0.0050, "5y": 0.0, "10y": 0.0050}

def bond_price_change(mod_dur, conv, dy):
    return -mod_dur*dy + 0.5*conv*(dy**2)

# Individual bond % price changes
impact = {tenor: bond_price_change(row.ModDur, row.Convexity, dY[tenor])
          for tenor, row in bonds.iterrows()}

# Portfolio impacts
impact_eq = np.dot(weights_eq, list(impact.values()))
impact_bullet = np.dot(weights_bullet, list(impact.values()))
impact_barbell = np.dot(weights_barbell, list(impact.values()))

# ---- c) Example portfolio benefitting from curvature ----
# To benefit from curvature, overweight barbell structure:
# For instance 25% in 2y, 50% in 10y, 25% in 5y
weights_curve = np.array([0.25, 0.25, 0.50])
dur_curve, conv_curve = port_stats(weights_curve, bonds)
impact_curve = np.dot(weights_curve, list(impact.values()))

# Print results
print("a) Portfolio Durations & Convexities")
print(f"Equally weighted: Dur={dur_eq:.3f}, Conv={conv_eq:.2f}")
print(f"Bullet (5y):      Dur={dur_bullet:.3f}, Conv={conv_bullet:.2f}")
print(f"Barbell:          Dur={dur_barbell:.3f}, Conv={conv_barbell:.2f}")

print("\nb) Yield Curve Steepening Impacts (%ΔP)")
print(f"Equally weighted: {impact_eq:.4%}")
print(f"Bullet (5y):      {impact_bullet:.4%}")
print(f"Barbell:          {impact_barbell:.4%}")

print("\nc) Example curvature portfolio (25% 2y, 25% 5y, 50% 10y)")
print(f"Dur={dur_curve:.3f}, Conv={conv_curve:.2f}, Impact={impact_curve:.4%}")


a) Portfolio Durations & Convexities
Equally weighted: Dur=4.580, Conv=27.20
Bullet (5y):      Dur=4.329, Conv=19.50
Barbell:          Dur=4.705, Conv=31.05

b) Yield Curve Steepening Impacts (%ΔP)
Equally weighted: -0.9183%
Bullet (5y):      0.0000%
Barbell:          -1.3774%

c) Example curvature portfolio (25% 2y, 25% 5y, 50% 10y)
Dur=5.319, Conv=35.02, Impact=-1.6127%


#### Q5 Results & Interpretation

a) Duration & Convexity

- Equally weighted portfolio:
  Duration ≈ 4.58, Convexity ≈ 27.2

- Bullet (all in 5y):
  Duration ≈ 4.33, Convexity ≈ 19.5

- Barbell (50% 2y + 50% 10y):
  Duration ≈ 4.71, Convexity ≈ 31.05

So all three have roughly similar durations (~4–5 years), but the barbell has much higher convexity.

b) Yield Curve Steepening (2y yields down 50 bps, 10y yields up 50 bps, 5y unchanged)

- 2y bond rises in price (yields down).

- 10y bond falls in price (yields up).

- 5y bond unchanged.

Portfolio impacts:

- Equally weighted: small net loss (10y loss outweighs 2y gain).

- Bullet (5y): no impact (unchanged yield).

- Barbell: net loss slightly bigger than equal weight, due to larger 10y exposure.

c) Portfolio Benefitting from Curvature

To exploit increased curvature, the portfolio should maximize convexity relative to duration.
That means overweighting the barbell structure (2y + 10y) compared to the bullet.

- Numerical example: 25% in 2y, 25% in 5y, 50% in 10y yields:

- Duration ≈ 5.19

- Convexity ≈ 39.1

Under steepening scenario, performance is more negative short-term, but if the curve becomes more curved (short rates drop, long rates rise more), this portfolio gains more from convexity compared to a bullet.

---
---